In [ ]:
# -*- coding: utf-8 -*-
# 目的：從 Supabase 抓取 6h 原始表，時間排序、補齊 6h 間隔，存為 Parquet

import os
from pathlib import Path
from typing import List, Dict, Any

import pandas as pd
from dotenv import load_dotenv
from supabase import create_client, Client

# ========== 0) 參數 ==========
TABLES = [
    "clean_price_ohlcv_6h",
    "clean_agg_oi_6h",
    "clean_funding_6h",
    "clean_liq_agg_6h",
    "clean_lsr_6h",
    "clean_taker_flow_6h",
]
OUTPUT_DIR = Path("./data/parquet_raw_6h")
PAGE_SIZE = 1000  # Supabase REST 單次返回上限；用 range 迭代抓全量
TIME_COL = "ts_utc"     # 對齊時間戳
TS_UTC_COL = "ts_utc" # 上載時間戳（若無則略過）

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# ========== 1) 連線 ==========
load_dotenv()
SUPABASE_URL = os.environ["SUPABASE_URL"]
SUPABASE_KEY = os.environ["SUPABASE_KEY"]
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# ========== 2) 通用抓取（含分頁） ==========
def fetch_all_rows(table: str, order_by: str = TIME_COL, page_size: int = PAGE_SIZE) -> List[Dict[str, Any]]:
    out: List[Dict[str, Any]] = []
    start = 0
    while True:
        resp = (
            supabase.table(table)
            .select("*")
            .order(order_by, desc=False)
            .range(start, start + page_size - 1)
            .execute()
        )
        rows = resp.data or []
        out.extend(rows)
        if len(rows) < page_size:
            break
        start += page_size
    return out

# ========== 3) 整理時間序並補齊 6H ==========
def normalize_df(df: pd.DataFrame, time_col: str = TIME_COL) -> pd.DataFrame:
    if df.empty:
        return df
    # 時間戳轉為 UTC DateTime
    # 支援秒或毫秒級 Unix（自動判斷）
    ts = pd.to_datetime(df[time_col], unit="ms", utc=True, errors="coerce")
    if ts.isna().all():  # 嘗試秒級
        ts = pd.to_datetime(df[time_col], unit="s", utc=True, errors="coerce")
    df = df.copy()
    df[time_col] = ts
    if TS_UTC_COL in df.columns:
        tsu = pd.to_datetime(df[TS_UTC_COL], unit="ms", utc=True, errors="coerce")
        if tsu.isna().all():
            tsu = pd.to_datetime(df[TS_UTC_COL], unit="s", utc=True, errors="coerce")
        df[TS_UTC_COL] = tsu
    # 設 index 與排序
    df = df.set_index(time_col).sort_index()
    # 以現有範圍補齊 6H 間隔
    full_idx = pd.date_range(start=df.index.min(), end=df.index.max(), freq="6H", tz="UTC")
    df = df.reindex(full_idx)
    # 不做值填補，保留 NaN 以供後續對齊（或可使用 ffill 視需求）
    df.index.name = time_col
    return df

# ========== 4) 主程式 ==========
for tbl in TABLES:
    rows = fetch_all_rows(tbl)
    df = pd.DataFrame(rows)
    if df.empty:
        print(f"[WARN] {tbl} 無資料，略過")
        continue
    df = normalize_df(df, TIME_COL)
    out_path = OUTPUT_DIR / f"{tbl}.parquet"
    df.to_parquet(out_path, index=True)
    print(f"[OK] {tbl} -> {out_path}")
